In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow.math as tm
import numpy as np
import time
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import initializers
from tensorflow.keras import Model
from tensorflow.keras import models
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

In [2]:
def convert2_zero_one(x):
    
    t = [tf.math.sigmoid(i) for i in x]    
    return t

def cont_bern_log_norm(lam, l_lim=0.49, u_lim=0.51):
    '''
    computes the log normalizing constant of a continuous Bernoulli distribution in a numerically stable way.
    returns the log normalizing constant for lam in (0, l_lim) U (u_lim, 1) and a Taylor approximation in
    [l_lim, u_lim].
    cut_y below might appear useless, but it is important to not evaluate log_norm near 0.5 as tf.where evaluates
    both options, regardless of the value of the condition.
    '''
    
    cut_lam = tf.where(tm.logical_or(tm.less(lam, l_lim), tm.greater(lam, u_lim)), lam, l_lim * tf.ones_like(lam))
    log_norm = tm.log(tm.abs(2.0 * tm.atanh(1 - 2.0 * cut_lam))) - tm.log(tm.abs(1 - 2.0 * cut_lam))
    taylor = tm.log(2.0) + 4.0 / 3.0 * tm.pow(lam - 0.5, 2) + 104.0 / 45.0 * tm.pow(lam - 0.5, 4)
    return tf.where(tm.logical_or(tm.less(lam, l_lim), tm.greater(lam, u_lim)), log_norm, taylor)

In [3]:
class StochasticMLP(Model):
    
    def __init__(self, hidden_layer_sizes=[100], n_outputs=10, lr=1e-3):
        super(StochasticMLP, self).__init__()
        self.hidden_layer_sizes = hidden_layer_sizes
        self.fc_layers = [Dense(layer_size) for layer_size in hidden_layer_sizes]
        self.output_layer = Dense(n_outputs)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
        
    def call(self, x):
        
        network = []
        
        for i, layer in enumerate(self.fc_layers):
            
            logits = layer(x)
            x = tfp.distributions.Bernoulli(logits=logits).sample()
            network.append(x)

        final_logits = self.output_layer(x) # initial the weight of output layer
            
        return network
    
    def target_log_prob(self, x, h, y, is_gibbs = False, is_hmc = False):
        
        # get current state
        if is_hmc:
            h_current = tf.split(h, self.hidden_layer_sizes, axis = 1)
        else:    
            h_current = [tf.cast(h_i, dtype=tf.float32) for h_i in h]
        h_current = convert2_zero_one(h_current)
        h_previous = [x] + h_current[:-1]
    
        nlog_prob = 0. # negative log probability
        
        for i, (cv, pv, layer) in enumerate(zip(h_current, h_previous, self.fc_layers)):
            
            logits = layer(pv)
            ce = tf.nn.sigmoid_cross_entropy_with_logits(labels = cv, logits = logits)
            if not is_gibbs:
                ce += cont_bern_log_norm(tf.nn.sigmoid(logits))
            
            nlog_prob += tf.reduce_sum(ce, axis = -1)
        
        fce = tf.nn.sigmoid_cross_entropy_with_logits(
            labels=tf.cast(y, tf.float32), logits=self.output_layer(h_current[-1]))
        nlog_prob += tf.reduce_sum(fce, axis = -1)
            
        return -1 * nlog_prob
    
    def gibbs_new_state(self, x, h, y):
        
        '''
            generate a new state for the network node by node in Gibbs setting.
        '''
        
        h_current = h
        h_current = [tf.cast(h_i, dtype=tf.float32) for h_i in h_current]
        
        in_layers = self.fc_layers
        out_layers = self.fc_layers[1:] + [self.output_layer]
        
        prev_vals = [x] + h_current[:-1]
        curr_vals = h_current
        next_vals = h_current[1:] + [y]
        
        for i, (in_layer, out_layer, pv, cv, nv) in enumerate(zip(in_layers, out_layers, prev_vals, curr_vals, next_vals)):

            # node by node
            
            nodes = tf.transpose(cv)
            prob_parents = tm.sigmoid(in_layer(pv))
            
            out_layer_weights = out_layer.get_weights()[0]
            
            next_logits = out_layer(cv)
            
            new_layer = []
            
            for j, node in enumerate(nodes):
                
                # get info for current node (i, j)
                
                prob_parents_j = prob_parents[:, j]
                out_layer_weights_j = out_layer_weights[j]
                
                # calculate logits and logprob for node is 0 or 1
                next_logits_if_node_0 = next_logits[:, :] - node[:, None] * out_layer_weights_j[None, :]
                next_logits_if_node_1 = next_logits[:, :] + (1 - node[:, None]) * out_layer_weights_j[None, :]
                
                #print(next_logits_if_node_0, next_logits_if_node_1)
                
                logprob_children_if_node_0 = -1 * tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(
                    labels=tf.cast(nv, dtype = tf.float32), logits=next_logits_if_node_0), axis = -1)
                
                logprob_children_if_node_1 = -1 * tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(
                    labels=tf.cast(nv, dtype = tf.float32), logits=next_logits_if_node_1), axis = -1)
                
                # calculate prob for node (i, j)
                prob_0 = (1 - prob_parents_j) * tm.exp(logprob_children_if_node_0)
                prob_1 = prob_parents_j * tm.exp(logprob_children_if_node_1)
                prob_j = prob_1 / (prob_1 + prob_0)
            
                # sample new state with prob_j for node (i, j)
                new_node = tfp.distributions.Bernoulli(probs = prob_j).sample() # MAY BE SLOW
                
                # update nodes and logits for following calculation
                new_node_casted = tf.cast(new_node, dtype = "float32")
                next_logits = next_logits_if_node_0 * (1 - new_node_casted)[:, None] \
                            + next_logits_if_node_1 * new_node_casted[:, None] 
                
                # keep track of new node values (in prev/curr/next_vals and h_new)
                new_layer.append(new_node)
           
            new_layer = tf.transpose(new_layer)
            h_current[i] = new_layer
            prev_vals = [x] + h_current[:-1]
            curr_vals = h_current
            next_vals = h_current[1:] + [y]
        
        return h_current
    
    def generate_hmc_kernel(self, x, y, step_size = pow(1000, -1/4)):
        
        adaptive_hmc = tfp.mcmc.SimpleStepSizeAdaptation(tfp.mcmc.HamiltonianMonteCarlo(
            target_log_prob_fn = lambda v: self.target_log_prob(x, v, y, is_hmc = True),
            num_leapfrog_steps = 2,
            step_size = step_size),
            num_adaptation_steps=int(100 * 0.8))
        
        return adaptive_hmc
    
    # new proposing-state method with HamiltonianMonteCarlo
    def propose_new_state_hamiltonian(self, x, h, y, hmc_kernel, is_update_kernel = True):
    
        h_current = h
        h_current = [tf.cast(h_i, dtype=tf.float32) for h_i in h_current]
        h_current = tf.concat(h_current, axis = 1)

        # run the chain (with burn-in)
        num_burnin_steps = 0
        num_results = 1

        samples = tfp.mcmc.sample_chain(
            num_results = num_results,
            num_burnin_steps = num_burnin_steps,
            current_state = h_current, # may need to be reshaped
            kernel = hmc_kernel,
            trace_fn = None,
            return_final_kernel_results = True)
    
        # Generate new states of chains
        #h_state = rerange(samples[0][0])
        h_state = samples[0][0]
        h_new = tf.split(h_state, self.hidden_layer_sizes, axis = 1) 
        
        # Update the kernel if necesssary
        if is_update_kernel:
            new_step_size = samples[2].new_step_size.numpy()
            ker_new = self.generate_hmc_kernel(x, y, new_step_size)
            return(h_new, ker_new)
        else:
            return h_new
    
    def update_weights(self, x, h, y, is_gibbs = False):
        
        with tf.GradientTape() as tape:
            loss = -1 * tf.reduce_mean(self.target_log_prob(x, h, y, is_gibbs = is_gibbs))
        
        grads = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
    
    def get_predictions(self, x):

        logits = 0.0
        for layer in self.fc_layers:
            logits = layer(x)
            x = tm.sigmoid(logits)
        
        logits = self.output_layer(x)
        probs = tm.sigmoid(logits)
        labels = tf.cast(tm.greater(probs, 0.5), tf.int32)

        return labels
    
    def get_loss(self, x, y):
        
        logits = 0.0
        for layer in self.fc_layers:
            logits = layer(x)
            x = tm.sigmoid(logits)
            
        logits = self.output_layer(x)
        loss = tf.nn.sigmoid_cross_entropy_with_logits(labels = tf.cast(y, tf.float32), logits = logits)
        
        return tf.reduce_sum(loss, axis = -1)

In [4]:
def standard_backprop(size, dat_train, dat_val, epochs):
    '''
    Standard Backpropogation training
    '''
    
    batch_size = 4
    
    print("Start Standard Backprop")
    model = keras.Sequential(
        [
            layers.InputLayer(input_shape=(2,)),
            layers.Dense(size, activation = "sigmoid"),
            layers.Dense(1, activation = "sigmoid")
        ]
    )   
    opt = tf.keras.optimizers.Adam(learning_rate=0.01)
    st = time.time()
    model.compile(loss = "binary_crossentropy", optimizer = opt, metrics = ["accuracy"])
    history = model.fit(dat_train, batch_size = batch_size, epochs = epochs, validation_data = dat_val)
    train_time = time.time() - st
    
    return train_time, history

In [5]:
def hmc(size, dat_train, dat_val, epochs, burnin = 500):
    '''
    HMC training
    '''
    # Setting
    # Get train labels and val labels
    target_train = np.concatenate([target for data, target in dat_train.as_numpy_iterator()])
    target_val = np.concatenate([target for data, target in dat_val.as_numpy_iterator()])
    
    print("Start HMC")
    model = StochasticMLP(hidden_layer_sizes = [size], n_outputs = 1, lr = 0.01)
    network = [model.call(data) for data, target in dat_train]
    kernels = [model.generate_hmc_kernel(data, target) for data, target in dat_train]  
    
    # Burnin
    print("Start HMC Burning")
    burnin_losses = []
    for i in range(burnin):
        
        if(i % 100 == 0): print("Step %d" % i)

        res = []
        burnin_loss = 0.0
        for bs, (data, target) in enumerate(dat_train):
            res.append(model.propose_new_state_hamiltonian(data, network[bs], target, kernels[bs]))
            burnin_loss += -1 * tf.reduce_mean(model.target_log_prob(data, network[bs], target))
    
        network, kernels = zip(*res)
        burnin_losses.append(burnin_loss / (bs + 1))
    
    # Training
    print("Start HMC Training")
    train_losses = []
    train_accs = []
    val_losses = []
    val_accs = []
    start_time = time.time()
    
    for epoch in range(epochs):
        
        # train
        for bs, (data, target) in enumerate(dat_train):
        
            model.update_weights(data, network[bs], target)
            network = [model.propose_new_state_hamiltonian(x, net, y, ker, is_update_kernel = False) \
                       for (x, y), net, ker in zip(dat_train, network, kernels)]
            
        train_loss = 0.0
        for data, target in dat_train:
            train_loss += tf.reduce_mean(model.get_loss(data, target))
        train_loss /= (bs + 1)
        train_losses.append(train_loss)       
        
        train_preds = [model.get_predictions(data) for data, target in dat_train]
        train_acc = accuracy_score(np.concatenate(train_preds), target_train)
        train_accs.append(train_acc)        
        
        # validate
        
        val_loss = 0.0
        for bs, (data, target) in enumerate(dat_val):
            val_loss += tf.reduce_mean(model.get_loss(data, target))
        val_loss /= (bs + 1)
        val_losses.append(val_loss)  
        
        val_preds = [model.get_predictions(data) for data, target in dat_val]
        val_acc = accuracy_score(np.concatenate(val_preds), target_val)
        val_accs.append(val_acc)
        
        print("Epoch %d/%d: - %.4fs/step - train_loss: %.4f - train_acc: %.4f - val_loss: %.4f - val_acc: %.4f" 
            % (epoch + 1, epochs, (time.time() - start_time) / (epoch + 1), train_loss, train_acc, val_loss, val_acc))

    train_time = time.time() - start_time
    return burnin_losses, train_time, {"train_acc": train_accs, "train_loss": train_losses,
                             "val_acc": val_accs, "val_loss": val_losses}

In [6]:
def gibbs(size, dat_train, dat_val, epochs, burnin = 500):
    '''
    Gibbs Training
    '''
    # Setting
    # Get train labels and val labels
    target_train = np.concatenate([target for data, target in dat_train.as_numpy_iterator()])
    target_val = np.concatenate([target for data, target in dat_val.as_numpy_iterator()])
    
    print("Start Gibbs")
    model = StochasticMLP(hidden_layer_sizes = [size], n_outputs=1, lr = 0.01)
    network = [model.call(data) for data, target in dat_train]
    
    # Burnin
    print("Start Gibbs Burning")    
    burnin_losses = []
    for i in range(burnin):
    
        if(i % 100 == 0): print("Step %d" % i)

        res = []
        burnin_loss = 0.0
        for bs, (data, target) in enumerate(dat_train):
            res.append(model.gibbs_new_state(data, network[bs], target))
            burnin_loss += -1 * tf.reduce_mean(model.target_log_prob(data, network[bs], target, is_gibbs = True))
            
        network = res
        burnin_losses.append(burnin_loss / (bs + 1))
    
    # Training
    train_losses = []
    train_accs = []
    val_losses = []
    val_accs = []
    start_time = time.time()
    
    for epoch in range(epochs):
        
        # train
        for bs, (data, target) in enumerate(dat_train):
        
            model.update_weights(data, network[bs], target, is_gibbs = True)
            network = [model.gibbs_new_state(x, net, y) for (x, y), net in zip(dat_train, network)]
            
        train_loss = 0.0
        for data, target in dat_train:
            train_loss += tf.reduce_mean(model.get_loss(data, target))
        train_loss /= (bs + 1)
        train_losses.append(train_loss)       
        
        train_preds = [model.get_predictions(data) for data, target in dat_train]
        train_acc = accuracy_score(np.concatenate(train_preds), target_train)
        train_accs.append(train_acc)        
        
        # validate
        
        val_loss = 0.0
        for bs, (data, target) in enumerate(dat_val):
            val_loss += tf.reduce_mean(model.get_loss(data, target))
        val_loss /= (bs + 1)
        val_losses.append(val_loss)  
        
        val_preds = [model.get_predictions(data) for data, target in dat_val]
        val_acc = accuracy_score(np.concatenate(val_preds), target_val)
        val_accs.append(val_acc)
        
        print("Epoch %d/%d: - %.4fs/step - train_loss: %.4f - train_acc: %.4f - val_loss: %.4f - val_acc: %.4f" 
            % (epoch + 1, epochs, (time.time() - start_time) / (epoch + 1), train_loss, train_acc, val_loss, val_acc))

    train_time = time.time() - start_time
    return burnin_losses, train_time, {"train_acc": train_accs, "train_loss": train_losses,
                             "val_acc": val_accs, "val_loss": val_losses}

In [7]:
np.random.seed(1234)
X, Y = make_moons(200, noise = 0.3)

# Split into test and training data
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2, random_state=73)
y_train = y_train.reshape(-1, 1)
y_val = y_val.reshape(-1, 1)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)
val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(32)

2022-03-07 22:58:47.580464: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
# running multiple times
N = 5
size = 32
epochs = 100
burnin = 100

res_bp, res_hmc, res_gibbs = [], [], []

for i in range(N):
    
    time_bp, history_bp = standard_backprop(size, train_ds, val_ds, epochs)
    burnin_loss_hmc, time_hmc, history_hmc = hmc(size, train_ds, val_ds, epochs, burnin)
    burnin_loss_gibbs, time_gibbs, history_gibbs = gibbs(size, train_ds, val_ds, epochs, burnin)
    
    hist_bp = {"train_acc": history_bp.history['accuracy'], "train_loss": history_bp.history['loss'], 
               "val_acc": history_bp.history['val_accuracy'], "val_loss": history_bp.history['val_loss']}
    rbp = {'time': time_bp, 'history': hist_bp}
    rhmc = {'time': time_hmc, 'burnin': burnin_loss_hmc, 'history': history_hmc}
    rgibbs = {'time': time_gibbs, 'burnin': burnin_loss_gibbs, 'history': history_gibbs}
    
    res_bp.append(rbp)
    res_hmc.append(rhmc)
    res_gibbs.append(rgibbs)

Start Standard Backprop
Epoch 1/100
1/5 [=====>........................] - ETA: 1s - loss: 0.6554 - accuracy: 0.6250

2022-03-07 22:58:49.730220: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


5/5 [==============================] - 0s 26ms/step - loss: 0.6698 - accuracy: 0.5312 - val_loss: 0.6757 - val_accuracy: 0.5750
Epoch 2/100
5/5 [==============================] - 0s 3ms/step - loss: 0.6042 - accuracy: 0.7937 - val_loss: 0.6447 - val_accuracy: 0.6750
Epoch 3/100
5/5 [==============================] - 0s 3ms/step - loss: 0.5811 - accuracy: 0.7625 - val_loss: 0.6347 - val_accuracy: 0.6000
Epoch 4/100
5/5 [==============================] - 0s 3ms/step - loss: 0.5389 - accuracy: 0.8125 - val_loss: 0.6381 - val_accuracy: 0.5750
Epoch 5/100
5/5 [==============================] - 0s 3ms/step - loss: 0.5042 - accuracy: 0.8438 - val_loss: 0.6367 - val_accuracy: 0.5750
Epoch 6/100
5/5 [==============================] - 0s 3ms/step - loss: 0.4719 - accuracy: 0.8562 - val_loss: 0.6227 - val_accuracy: 0.5750
Epoch 7/100
5/5 [==============================] - 0s 3ms/step - loss: 0.4427 - accuracy: 0.8438 - val_loss: 0.6115 - val_accuracy: 0.6000
Epoch 8/100
5/5 [=====================

5/5 [==============================] - 0s 3ms/step - loss: 0.3262 - accuracy: 0.8750 - val_loss: 0.5953 - val_accuracy: 0.7000
Epoch 60/100
5/5 [==============================] - 0s 3ms/step - loss: 0.3261 - accuracy: 0.8750 - val_loss: 0.5954 - val_accuracy: 0.7000
Epoch 61/100
5/5 [==============================] - 0s 3ms/step - loss: 0.3260 - accuracy: 0.8750 - val_loss: 0.5955 - val_accuracy: 0.7000
Epoch 62/100
5/5 [==============================] - 0s 3ms/step - loss: 0.3260 - accuracy: 0.8750 - val_loss: 0.5956 - val_accuracy: 0.7000
Epoch 63/100
5/5 [==============================] - 0s 3ms/step - loss: 0.3259 - accuracy: 0.8750 - val_loss: 0.5957 - val_accuracy: 0.7000
Epoch 64/100
5/5 [==============================] - 0s 3ms/step - loss: 0.3259 - accuracy: 0.8750 - val_loss: 0.5958 - val_accuracy: 0.7000
Epoch 65/100
5/5 [==============================] - 0s 3ms/step - loss: 0.3258 - accuracy: 0.8750 - val_loss: 0.5959 - val_accuracy: 0.7000
Epoch 66/100
5/5 [===============

Epoch 22/100: - 0.6659s/step - train_loss: 0.5850 - train_acc: 0.8438 - val_loss: 0.6358 - val_acc: 0.6000
Epoch 23/100: - 0.6668s/step - train_loss: 0.5826 - train_acc: 0.8500 - val_loss: 0.6335 - val_acc: 0.6000
Epoch 24/100: - 0.6670s/step - train_loss: 0.5794 - train_acc: 0.8562 - val_loss: 0.6305 - val_acc: 0.5750
Epoch 25/100: - 0.6675s/step - train_loss: 0.5763 - train_acc: 0.8500 - val_loss: 0.6277 - val_acc: 0.6250
Epoch 26/100: - 0.6676s/step - train_loss: 0.5735 - train_acc: 0.8500 - val_loss: 0.6258 - val_acc: 0.6250
Epoch 27/100: - 0.6674s/step - train_loss: 0.5701 - train_acc: 0.8562 - val_loss: 0.6240 - val_acc: 0.6000
Epoch 28/100: - 0.6674s/step - train_loss: 0.5661 - train_acc: 0.8625 - val_loss: 0.6217 - val_acc: 0.5750
Epoch 29/100: - 0.6674s/step - train_loss: 0.5626 - train_acc: 0.8562 - val_loss: 0.6196 - val_acc: 0.6000
Epoch 30/100: - 0.6676s/step - train_loss: 0.5599 - train_acc: 0.8562 - val_loss: 0.6172 - val_acc: 0.6000
Epoch 31/100: - 0.6678s/step - train_

...

Epoch 78/100: - 2.4718s/step - train_loss: 0.6442 - train_acc: 0.8313 - val_loss: 0.6552 - val_acc: 0.6500
Epoch 79/100: - 2.4717s/step - train_loss: 0.6460 - train_acc: 0.8688 - val_loss: 0.6577 - val_acc: 0.6750
Epoch 80/100: - 2.4719s/step - train_loss: 0.6455 - train_acc: 0.8812 - val_loss: 0.6578 - val_acc: 0.6750
Epoch 81/100: - 2.4725s/step - train_loss: 0.6453 - train_acc: 0.8500 - val_loss: 0.6562 - val_acc: 0.6500
Epoch 82/100: - 2.4738s/step - train_loss: 0.6444 - train_acc: 0.7125 - val_loss: 0.6550 - val_acc: 0.5750
Epoch 83/100: - 2.4746s/step - train_loss: 0.6441 - train_acc: 0.8500 - val_loss: 0.6568 - val_acc: 0.6250
Epoch 84/100: - 2.4767s/step - train_loss: 0.6418 - train_acc: 0.8688 - val_loss: 0.6581 - val_acc: 0.6500
Epoch 85/100: - 2.4779s/step - train_loss: 0.6372 - train_acc: 0.8688 - val_loss: 0.6548 - val_acc: 0.6500
Epoch 86/100: - 2.4792s/step - train_loss: 0.6365 - train_acc: 0.7812 - val_loss: 0.6526 - val_acc: 0.6000
Epoch 87/100: - 2.4808s/step - train_

Epoch 42/100
5/5 [==============================] - 0s 3ms/step - loss: 0.3258 - accuracy: 0.8750 - val_loss: 0.5979 - val_accuracy: 0.7000
Epoch 43/100
5/5 [==============================] - 0s 3ms/step - loss: 0.3258 - accuracy: 0.8750 - val_loss: 0.5979 - val_accuracy: 0.7000
Epoch 44/100
5/5 [==============================] - 0s 3ms/step - loss: 0.3257 - accuracy: 0.8750 - val_loss: 0.5979 - val_accuracy: 0.7000
Epoch 45/100
5/5 [==============================] - 0s 3ms/step - loss: 0.3257 - accuracy: 0.8750 - val_loss: 0.5979 - val_accuracy: 0.7000
Epoch 46/100
5/5 [==============================] - 0s 3ms/step - loss: 0.3256 - accuracy: 0.8750 - val_loss: 0.5979 - val_accuracy: 0.7000
Epoch 47/100
5/5 [==============================] - 0s 3ms/step - loss: 0.3255 - accuracy: 0.8750 - val_loss: 0.5979 - val_accuracy: 0.7000
Epoch 48/100
5/5 [==============================] - 0s 3ms/step - loss: 0.3255 - accuracy: 0.8750 - val_loss: 0.5979 - val_accuracy: 0.7000
Epoch 49/100
5/5 [==

5/5 [==============================] - 0s 3ms/step - loss: 0.3228 - accuracy: 0.8750 - val_loss: 0.6040 - val_accuracy: 0.6750
Start HMC
Start HMC Burning
Step 0
Start HMC Training
Epoch 1/100: - 0.7091s/step - train_loss: 0.6791 - train_acc: 0.5188 - val_loss: 0.6829 - val_acc: 0.4250
Epoch 2/100: - 0.7099s/step - train_loss: 0.6704 - train_acc: 0.8000 - val_loss: 0.6825 - val_acc: 0.6000
Epoch 3/100: - 0.7349s/step - train_loss: 0.6640 - train_acc: 0.8000 - val_loss: 0.6800 - val_acc: 0.5750
Epoch 4/100: - 0.7430s/step - train_loss: 0.6587 - train_acc: 0.7000 - val_loss: 0.6757 - val_acc: 0.5750
Epoch 5/100: - 0.7516s/step - train_loss: 0.6547 - train_acc: 0.6500 - val_loss: 0.6725 - val_acc: 0.5000
Epoch 6/100: - 0.7492s/step - train_loss: 0.6507 - train_acc: 0.6500 - val_loss: 0.6697 - val_acc: 0.5000
Epoch 7/100: - 0.7461s/step - train_loss: 0.6472 - train_acc: 0.6500 - val_loss: 0.6672 - val_acc: 0.5000
Epoch 8/100: - 0.7486s/step - train_loss: 0.6433 - train_acc: 0.6937 - val_lo

Epoch 76/100: - 0.7699s/step - train_loss: 0.4730 - train_acc: 0.8625 - val_loss: 0.5676 - val_acc: 0.6000
Epoch 77/100: - 0.7692s/step - train_loss: 0.4689 - train_acc: 0.8750 - val_loss: 0.5658 - val_acc: 0.6000
Epoch 78/100: - 0.7685s/step - train_loss: 0.4658 - train_acc: 0.8625 - val_loss: 0.5643 - val_acc: 0.6250
Epoch 79/100: - 0.7677s/step - train_loss: 0.4642 - train_acc: 0.8500 - val_loss: 0.5628 - val_acc: 0.6250
Epoch 80/100: - 0.7678s/step - train_loss: 0.4632 - train_acc: 0.8500 - val_loss: 0.5605 - val_acc: 0.6250
Epoch 81/100: - 0.7683s/step - train_loss: 0.4624 - train_acc: 0.8688 - val_loss: 0.5578 - val_acc: 0.6250
Epoch 82/100: - 0.7690s/step - train_loss: 0.4631 - train_acc: 0.8750 - val_loss: 0.5570 - val_acc: 0.6000
Epoch 83/100: - 0.7692s/step - train_loss: 0.4644 - train_acc: 0.8688 - val_loss: 0.5572 - val_acc: 0.6250
Epoch 84/100: - 0.7692s/step - train_loss: 0.4655 - train_acc: 0.8688 - val_loss: 0.5579 - val_acc: 0.6250
Epoch 85/100: - 0.7692s/step - train_

Epoch 53/100: - 2.4336s/step - train_loss: 0.6635 - train_acc: 0.8438 - val_loss: 0.6690 - val_acc: 0.6250
Epoch 54/100: - 2.4331s/step - train_loss: 0.6631 - train_acc: 0.6375 - val_loss: 0.6682 - val_acc: 0.5500
Epoch 55/100: - 2.4333s/step - train_loss: 0.6637 - train_acc: 0.5563 - val_loss: 0.6696 - val_acc: 0.5000
Epoch 56/100: - 2.4333s/step - train_loss: 0.6635 - train_acc: 0.8125 - val_loss: 0.6724 - val_acc: 0.6500
Epoch 57/100: - 2.4334s/step - train_loss: 0.6638 - train_acc: 0.8500 - val_loss: 0.6736 - val_acc: 0.6250
Epoch 58/100: - 2.4348s/step - train_loss: 0.6634 - train_acc: 0.8125 - val_loss: 0.6721 - val_acc: 0.6500
Epoch 59/100: - 2.4347s/step - train_loss: 0.6619 - train_acc: 0.7562 - val_loss: 0.6703 - val_acc: 0.6000
Epoch 60/100: - 2.4346s/step - train_loss: 0.6610 - train_acc: 0.8250 - val_loss: 0.6698 - val_acc: 0.6500
Epoch 61/100: - 2.4353s/step - train_loss: 0.6604 - train_acc: 0.8125 - val_loss: 0.6685 - val_acc: 0.6000
Epoch 62/100: - 2.4416s/step - train_

In [10]:
res_bp_tmp = []
for i in range(N):
    hist_bp = {"train_acc": res_bp[i]['history']['train_acc'], "train_loss": res_bp[i]['history']['train_loss'], 
               "val_acc": res_bp[i]['history']['val_acc'], "val_loss": res_bp[i]['history']['val_loss']}
    r_bp = {"time": res_bp[i]['time'], "history": hist_bp}
    res_bp_tmp.append(r_bp)

In [11]:
# plot all the running times for each method
def plot_all(res, method, metric):
    
    plt.style.use('seaborn')
    nrow = 3
    ncol = 2
    
    fig, ax = plt.subplots(nrow, ncol, sharex = True)
    fig.suptitle(method + "_" + metric)
    for i in range(nrow):
        for j in range(ncol):
            if i * ncol + j < N:
                ax[i, j].plot(res[i * ncol + j]['history'][metric])
                ax[i, j].set_title(f"Run {i * ncol + j}")
    plt.savefig(method + "_" + metric + '.pdf')
    plt.close()

res_bp = res_bp_tmp

res_all = [res_bp, res_hmc, res_gibbs]
methods = ['bp', 'hmc', 'gibbs']
metrics = ['train_acc', 'train_loss', 'val_acc', 'val_loss']
for i, method in enumerate(methods):
    for metric in metrics:
        plot_all(res_all[i], method, metric)

In [12]:
# calculate average curve for each method
def cal_avg(res):
    
    metrics = ['train_acc', 'train_loss', 'val_acc', 'val_loss']
    avg = {}
    for metric in metrics:
        arr_metric = np.zeros((N, epochs))
        for i in range(N):
            arr_metric[i] = np.array(res[i]['history'][metric])
        avg_metric = np.mean(arr_metric, axis = 0)
        avg[metric] = avg_metric
        
    return avg

avg_bp = cal_avg(res_bp)
avg_hmc = cal_avg(res_hmc)
avg_gibbs = cal_avg(res_gibbs)

In [13]:
avg_all = [avg_bp, avg_hmc, avg_gibbs]

plt.style.use('seaborn')
for metric in metrics:
    for i, method in enumerate(methods):
        plt.plot(avg_all[i][metric], label = method)
    plt.title(metric)
    plt.xlabel("epochs")
    plt.ylabel(metric.split("_")[1])
    plt.legend()
    plt.savefig("average_" + metric + '.pdf')
    plt.close()

In [14]:
time_bp, time_hmc, time_gibbs = [], [], []
for i in range(N):
    time_bp.append(res_bp[i]['time'])
    time_hmc.append(res_hmc[i]['time'])
    time_gibbs.append(res_gibbs[i]['time'])

In [15]:
with open('time.npy', 'wb') as f:
    np.save(f, np.array(res_all))
    np.save(f, np.array(time_bp))
    np.save(f, np.array(time_hmc))
    np.save(f, np.array(time_gibbs))